In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np

3blue1brown issued a challenge to all of us to understand why the collision of objects results in calculating $\pi$ (pi). The guy at Coding Train did it as one of his challenges, but while I was watching it, I was amazed by how accurate he was able to get it, considering that he used Euler integration. Finally it occurred to me that he wasn't tracking accurately at *all* where the objects were, at least not accurately enough to matter. 

His collision detection just checked whether the objects overlapped. So, in one step, the two blocks overlap, but they must overlap by less than the relative motion of the blocks over one step, because if it were more, they would have overlapped in the previous step. The collision and reflection is then guaranteed to get them to not overlap in the next step, because the blocks relative motion is reversed.

The more I thought about it, the more it became obvious that:

* *We don't have to track position at all* - because the small block is trapped. It is either moving towards the wall and will hit the wall (the big block will never move faster towards the wall than the small block), it is moving away from the wall faster than the big block, in which case it will hit it, or moving away from the wall slower than the big block, in which case it will never hit and we can stop counting.
* *We don't have to track time at all* - It is always obvious from the cases above what will happen next, and the physics are invariant over time and space so it doesn't matter when and where it happens.

So, we track the velocities of the two blocks, and every time through the loop, one of three things happens:

1. Small block moving towards the wall - reflect off the wall
2. Small block moving away from wall, faster than big block - reflect off the block
3. Small block moving away from wall, slower than big block - stop counting



As far as interactions, we will steal straight from [Wikipedia](https://en.wikipedia.org/wiki/Elastic_collision#One-dimensional_Newtonian), just like Coding Train did. I will quote them, just changing the block names so I can be more consistent with the variable names.

> Consider particles $A$ and $B$ with masses $m_A$, $m_B$, and velocities $v_{A0}$, $v_{B0}$ before collision,  $v_{A1}$, $v_{B1}$ after collision.<br /><br />
>The conservation of the total [momentum](https://en.wikipedia.org/wiki/Momentum) before and after the collision is expressed by:<br />
$$m_{A}v_{A0}+m_{B}v_{B0}=m_{A}v_{A1} + m_{B}v_{B1}.$$
>Likewise, the conservation of the total [kinetic energy](https://en.wikipedia.org/wiki/Kinetic_energy) is expressed by: $$\tfrac12 m_Av_{A0}^2+\tfrac12 m_Bv_{B0}^2=\tfrac12 m_Av_{A1}^2 +\tfrac12 m_{B}v_{B1}^2.$$
>These equations may be solved directly to find <math>v_1,v_2</math> when <math>u_1,u_2</math> are known: $$\begin{eqnarray*}
v_{A1} &=& \frac{m_A-m_B}{m_A+m_B}v_{A0} + \frac{2m_B}{m_A+m_B} v_{B0} \\
v_{B1} &=& \frac{2m_A}{m_A+m_B} v_{A0} + \frac{m_B-m_A}{m_A+m_B} v_{B0}
\end{eqnarray*}$$

In [ ]:
def collide(m_A,m_B,v_A0,v_B0):
    v_A1=(m_A-m_B)/(m_A+m_B)*v_A0+(  2*m_B)/(m_A+m_B)*v_B0
    v_B1=(  2*m_A)/(m_A+m_B)*v_A0+(m_B-m_A)/(m_A+m_B)*v_B0
    return (v_A1,v_B1)

>If both masses are the same, we have a trivial solution:$$\begin{eqnarray*}
v_{A1}&=&u_{B0}\\
v_{B1}&=&v_{A0}\end{eqnarray*}.$$
>This simply corresponds to the bodies exchanging their initial velocities to each other.<br /><br />
>As can be expected, the solution is invariant under adding a constant to all velocities, which is like using a frame of reference with constant translational velocity. Indeed, to derive the equations, one may first change the frame of reference so that one of the known velocities is zero, determine the unknown velocities in the new frame of reference, and convert back to the original frame of reference.<br /><br />
> **Examples**
> * Ball A: mass = 3 kg,       velocity =  4 m/s
> * Ball B: mass = 5 kg,       velocity = -6 m/s

In [ ]:
(v_A1,v_B1)=collide(m_A=3,m_B=5,v_A0=4,v_B0=-6)

> After collision:
> * Ball A: velocity = -8.5 m/s
> * Ball B: velocity =  1.5 m/s

In [ ]:
print("v_A1: %.1f    v_B1: %.1f"%(v_A1,v_B1))

One way to consider the collision with the wall is just to consider the wall to be immovable and just reverse the velocity of the small block upon collision. I instead am going to pursue the idea of using the exact same collision formula, but consider the wall to have infinite mass.

Note that the formula has an indeterminacy ($-\infty/\infty$) if we treat one mass as inifinite. What we will say instead is that the mass of the wall is so great that $m_A/m_B\approx 0$. Solving this for $m_A$ we get $m_A\approx 0m_B\approx 0$.

$$\require{cancel}
\begin{eqnarray*}
v_{A1} &=& \frac{\cancelto{0}{m_A}-m_B}{\cancelto{0}{m_A}+m_B}v_{A0} + \frac{2m_B}{\cancelto{0}{m_A}+m_B} v_{B0} \\
  &=& \frac{-m_B}{m_B}v_{A0} + \frac{2m_B}{m_B} v_{B0} \\
  &=& -v_{A0} + 2 v_{B0} \\
v_{B1} &=& \frac{2\cancelto{0}{m_A}}{\cancelto{0}{m_A}+m_B} v_{A0} + \frac{m_B-\cancelto{0}{m_A}}{\cancelto{0}{m_A}+m_B} v_{B0}\\
       &=& \frac{0}{m_B} v_{A0} + \frac{m_B}{m_B} v_{B0}\\
       &=& v_{B0}
\end{eqnarray*}$$

Since the wall is immovable, it isn't moving, so $v_{B0}=0$. Plugging this in above shows that yes, the velocity of the block does reverse, and the velocity of the wall stays zero.

In [ ]:
print("v_A1: %.1f    v_B1: %.1f"%collide(m_A=0,m_B=1,v_A0=-1,v_B0=0))

Now we reach the loop. Since we don't know ahead of time how many times we will loop (shut up, $\pi$), we will use a `while not done:` loop. Inside the loop, we will consider a tree of cases:

* If the small block is moving towards the wall
  - If the big block is moving towards the wall faster than the small block:
    - Throw an exception. We would have to check whether the small block collides with the wall first or the big block overtakes and collides with the small block first. If this ever happens, it throws away our critical assumption that we don't have to track the block positions.
  - Else
    - Collide the small block with the wall
* Else the small block is moving away from the wall (or stopped)
  - If the big block is moving away from the wall faster than the small one:
    - We are done - the blocks will never collide again with anything
  - Else
    - Collide the blocks

In [ ]:
done=False
digits=8
m_A=1
m_B=m_A*100**(digits-1)
v_A=0
v_B=-1
i=0
print("m_A: %f    m_B: %f"%(m_A,m_B))
#print(("%"+str(digits)+"d   v_A: %f    v_B: %f")%(i,v_A,v_B))
while not done:
    if v_A<0:
        #if v_B<v_A:
        #    raise ValueError("This really does happen, I guess")
        #else:
            #(v_A,_)=collide(0,1,v_A,0) #Collide with wall
        v_A=-v_A
    else:
        if v_B>v_A:
            done=True
            print("Done")
        else:
            (v_A,v_B)=collide(m_A,m_B,v_A,v_B)
    if not done:
        i+=1
        #print(("%"+str(digits)+"d   v_A: %f    v_B: %f")%(i,v_A,v_B))
        if(i%1000000)==0:
            print(("%0"+str(digits)+"d")%i)
print(i)
        

So that works, at least to eight digits. I expect it to be good to the significant figures of double precision, and I also expect that it will take ten to the eighth times as long to run that far.

Now, what if we do run a physics engine? What if we do track positions? What if we want to see how fast the blocks are going and how many clacks per second we reach? 

We can just consider the blocks as point masses -- equivalently, we will consider that if the left edge of the small block is touching the wall, the right edge is at the origin, and therefore the wall is one small block width to the left of the origin. We will track the right edge of the small block and the left edge of the big block.

I can talk myself into an argument (I won't dignify it with the word "proof") that the try case can never happen, because it isn't possible for the small block to be moving slower than the big block while the big block is moving towards the wall. So, we will have the following cases:

* If the small block is moving towards the wall:
  - use the position of the block at the last clack and the distance to the wall to figure the time to hit the wall, then advance the clock that far, set the small block position to zero, track the big block position, and reverse the small block speed.$$\begin{eqnarray*}
  \Delta t&=&-\frac{x_{Ai}}{v_{Ai}} \\
  t_{i+1}&=&t_i+\Delta t \\
  x_{Ai+1}&=&0 \\
  x_{Bi+1}&=&x_{Bi}+\Delta t v_{Bi}  \\
  v_{Bi+1}&=&v_{Bi} \\
  v_{Ai+1}&=&-v_{Ai} \\
\end{eqnarray*}$$
* Else if both blocks are moving away from the wall, but the small block is slower than the big block:
  - We are done. Report the final state
* Else the small block will hit the big block:
  - Use the position and speeds of both blocks to figure out when the blocks will hit. Since both blocks are moving linearly, this is where the lines intersect.$$\begin{eqnarray*}
  x_{Ai}+\Delta t v_{Ai}&=&x_{Bi}+\Delta t v_{Bi}\\
  \Delta t v_{Ai}-\Delta t v_{Bi}&=&x_{Bi}-x_{Ai}\\
  \Delta t(v_{Ai}-v_{Bi})&=&x_{Bi}-x_{Ai}\\
  \Delta t&=&\frac{x_{Bi}-x_{Ai}}{v_{Ai}-v_{Bi}}\\
\end{eqnarray*}$$
  - Use the normal collide() function to determine speed after the collision$$\begin{eqnarray*}
  (v_{Ai+1},v_{Bi+1})&=&\operatorname{collide}(m_A,m_B,v_{Ai},v_{Bi}) \\
\end{eqnarray*}$$
  - Update the time and position$$\begin{eqnarray*}
  t_{i+1}&=&t_i+\Delta t \\
  x_{Ai+1}&=&x_{Ai}+\Delta t v_{Ai} \\
  x_{Bi+1}&=&x_{Bi}+\Delta t v_{Bi} (\mbox{ should}=x_{Ai+1}) \\
\end{eqnarray*}$$


In [ ]:
done=False
digits=8
m_A=1
m_B=m_A*100**(digits-1)
x_A=1
x_B=2
v_A=0
v_B=-1
i=0
t=0
ts=[0]
v_As=[v_A]
min_dt=float('inf')
max_Va=-float('inf')
min_Xb=float('inf')
print("m_A: %e    m_B: %e"%(m_A,m_B))
print(("0%"+str(digits)+"d t: %e    x_A: %e    v_A: %e    x_B: %e  v_B: %e")%(i,t,x_A,v_A,x_B,v_B))
while not done:
    if v_A<0:
        dt=-x_A/v_A
        t=t+dt
        x_A=0
        x_B=x_B+dt*v_B
        v_A=-v_A
    else:
        if v_B>v_A:
            done=True
            print("Done")
        else:
            dt=(x_B-x_A)/(v_A-v_B)
            x_A=x_A+dt*v_A
            x_B=x_B+dt*v_B
            (v_A,v_B)=collide(m_A,m_B,v_A,v_B)
            t=t+dt
    if(v_A>max_Va):
        max_Va=v_A
    if(dt<min_dt):
        min_dt=dt
    if(x_B<min_Xb):
        min_Xb=x_B
    if not done:
        i+=1
        ts.append(t)
        v_As.append(v_A)
        if(i%10**(digits-2))==0:
            print(("%0"+str(digits)+"d t: %e    x_A: %e    v_A: %e    x_B: %e  v_B: %e")%(i,t,x_A,v_A,x_B,v_B))
print("%d %e %e %e %e"%(i,min_Xb,min_dt,1.0/min_dt,max_Va))
plt.figure("Speed of block A")
plt.plot(np.array(ts),np.abs(np.array(v_As)))
plt.show()

Evidence suggests that as the mass increases:
* The minimum distance of block B to the wall is $k\sqrt{\frac{m_B}{m_A}}$
* The maximum speed of block A is $|v_{B0}\sqrt{\frac{m_B}{m_A}}|$,
* 